4.4 Serialization and Composition

1. How to seperate the prompt from the code


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

from langchain.prompts import load_prompt

#prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

prompt.format(country="Vietnam")

'what is the capital of Vietnam'

2. How to compose the prompts altogether

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts=[ # leftone should be same with the template name
    ("intro", intro),
    ("example", example),
    ("start", start)
]


full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)
# full_prompt.format(
#     character="Suspicious Mafia pretend to be a normal citizen",
#     example_question="What did you do last night",
#     example_answer = """
#     Oh, just a quiet evening at home. 
#     Just me, a glass of wine, and a little... paperwork to sort out. 
#     Nothing too exciting—just the usual stuff. 
#     How about you? Anything interesting?:)
#     """,
#     question="I think you are a bit suspicious. what is your secret?"
# )

chain = full_prompt | chat

chain.invoke({
    "character":"Suspicious Mafia pretend to be a normal citizen",
    "example_question":"What did you do last night",
    "example_answer" : """
    Oh, just a quiet evening at home. 
    Just me, a glass of wine, and a little... paperwork to sort out. 
    Nothing too exciting—just the usual stuff. 
    How about you? Anything interesting?:)
    """,
    "question":"I think you are a bit suspicious. what is your secret?"
})



Oh, me? Suspicious? No way, I'm just a regular citizen going about my business. 
No secrets here, just trying to keep a low profile and stay out of trouble. 
But hey, if you've got any juicy gossip or secrets to share, I'm all ears. 
Let's keep things interesting, shall we? ;)

AIMessage(content="Oh, me? Suspicious? No way, I'm just a regular citizen going about my business. \nNo secrets here, just trying to keep a low profile and stay out of trouble. \nBut hey, if you've got any juicy gossip or secrets to share, I'm all ears. \nLet's keep things interesting, shall we? ;)", response_metadata={'finish_reason': 'stop'}, id='run-9fdd8333-0b5d-4eda-9549-9e35ba47273d-0')

Caching can save LLM's Response.

Ex) If you build a chat bot always answer same question

https://python.langchain.com/v0.1/docs/modules/model_io/llms/llm_caching/
https://python.langchain.com/v0.1/docs/integrations/llms/llm_caching/

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug 
from langchain.cache import InMemoryCache, SQLiteCache 


#set_llm_cache(InMemoryCache()) # all response will be saved in memory.
set_llm_cache(SQLiteCache("myCache.db")) ## all response will be saved in DB

set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chat.predict("How do you sleep fast") # before caching -6s



[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you sleep fast"
  ]
}
There are several techniques that can help you fall asleep faster:

1. Establish a bedtime routine: Try to go to bed and wake up at the same time every day to regulate your body's internal clock.

2. Create a relaxing sleep environment: Make sure your bedroom is dark, quiet, and cool. Consider using earplugs or a white noise machine to block out any distractions.

3. Limit screen time before bed: The blue light emitted by screens can interfere with your body's production of melatonin, a hormone that regulates sleep. Try to avoid screens for at least an hour before bedtime.

4. Practice relaxation techniques: Deep breathing, meditation, or progressive muscle relaxation can help calm your mind and body before bed.

5. Avoid caffeine and heavy meals close to bedtime: Stimulants like caffeine can interfere with your ability to fall asleep, while heavy meals can cause discomfo

"There are several techniques that can help you fall asleep faster:\n\n1. Establish a bedtime routine: Try to go to bed and wake up at the same time every day to regulate your body's internal clock.\n\n2. Create a relaxing sleep environment: Make sure your bedroom is dark, quiet, and cool. Consider using earplugs or a white noise machine to block out any distractions.\n\n3. Limit screen time before bed: The blue light emitted by screens can interfere with your body's production of melatonin, a hormone that regulates sleep. Try to avoid screens for at least an hour before bedtime.\n\n4. Practice relaxation techniques: Deep breathing, meditation, or progressive muscle relaxation can help calm your mind and body before bed.\n\n5. Avoid caffeine and heavy meals close to bedtime: Stimulants like caffeine can interfere with your ability to fall asleep, while heavy meals can cause discomfort and indigestion.\n\n6. Exercise regularly: Physical activity can help improve the quality of your slee

In [5]:
chat.predict("How do you sleep fast") #after caching -0s

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you sleep fast"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "There are several techniques that can help you fall asleep faster:\n\n1. Establish a bedtime routine: Try to go to bed and wake up at the same time every day to regulate your body's internal clock.\n\n2. Create a relaxing sleep environment: Make sure your bedroom is dark, quiet, and cool. Consider using earplugs or a white noise machine to block out any distractions.\n\n3. Limit screen time before bed: The blue light emitted by screens can interfere with your body's production of melatonin, a hormone that regulates sleep. Try to avoid screens for at least an hour before bedtime.\n\n4. Practice relaxation techniques: Deep breathing, meditation, or progressive muscle relaxation can help calm your mind and body before bed.\n\n5. Avoid caffeine and heavy meals close 

"There are several techniques that can help you fall asleep faster:\n\n1. Establish a bedtime routine: Try to go to bed and wake up at the same time every day to regulate your body's internal clock.\n\n2. Create a relaxing sleep environment: Make sure your bedroom is dark, quiet, and cool. Consider using earplugs or a white noise machine to block out any distractions.\n\n3. Limit screen time before bed: The blue light emitted by screens can interfere with your body's production of melatonin, a hormone that regulates sleep. Try to avoid screens for at least an hour before bedtime.\n\n4. Practice relaxation techniques: Deep breathing, meditation, or progressive muscle relaxation can help calm your mind and body before bed.\n\n5. Avoid caffeine and heavy meals close to bedtime: Stimulants like caffeine can interfere with your ability to fall asleep, while heavy meals can cause discomfort and indigestion.\n\n6. Exercise regularly: Physical activity can help improve the quality of your slee

4-6 How to check how much the calling the OpenAI tasks will costs

And How to save and load the model

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback # to check cost of LLM

chat = ChatOpenAI(
    temperature=0.9,
)

with get_openai_callback() as usage: #usage - name of callback
    a = chat.predict("How to be a better programmer without senior programmer mentor")
    b = chat.predict("How to be better at kickboxing")
    print(a, b, "/n")
    print(usage) #print(usage.total_cost )

/home/minji/privateGPT/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/minji/privateGPT/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


1. Take online courses and tutorials: There are plenty of resources available online for learning programming languages and improving your skills. Websites like Codecademy, Coursera, and Udemy offer courses on a wide range of programming topics.

2. Practice coding regularly: The more you practice coding, the better you will become. Set aside time each day to work on coding challenges, side projects, or open source contributions.

3. Read books and documentation: Books on programming concepts and best practices can provide valuable insights and help you deepen your understanding of programming languages. Reading the official documentation of the languages or technologies you work with can also be helpful.

4. Join programming communities: Joining online programming communities, such as Stack Overflow, Reddit programming subreddits, or Meetup groups, can allow you to connect with other programmers, share knowledge, and seek advice.

5. Collaborate with others: Working on projects with o

In [2]:
from langchain.llms.openai import OpenAI


chat = OpenAI(
    temperature=0.9,
    max_tokens=450,
    model="gpt-4"
)



chat.save("model.json")

/home/minji/privateGPT/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [4]:
from langchain.llms.loading import load_llm

chat = load_llm("model.json")

chat

/home/minji/privateGPT/env/lib/python3.10/site-packages/langchain_community/llms/openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/home/minji/privateGPT/env/lib/python3.10/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=APIRemovedInV1Proxy, model_name='gpt-4', model_kwargs={'temperature': 0.9, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 450})